In [1]:
LPNE_PATH = "./lpne-data-analysis/"
DATA_PATH = "/work/mk423/CLOCK_Apr_2024/"
LABELS_PATH = DATA_PATH + "labels/"
SAVE_FILE = "/work/mk423/CLOCK_Apr_2024/C19_EPM_Apr_2024_Data.pkl"
FEATURE_LIST = ["power","coherence","granger"]
import os, sys
sys.path.append(LPNE_PATH)
import numpy as np
import pandas as pd
import pickle
import data_tools

In [2]:
X_psd, X_coh, X_gc, labels = data_tools.load_data(DATA_PATH + "CLOCK_Apr_2024.mat", feature_list = ["power","coherence","granger"])

version saveFeatures_1.2 used to calcuate power features
version saveFeatures_1.2 used to calcuate coherence features
version saveFeatures_1.2 used to calcuate granger features
Version preprocessData_1.0 of preprocessing used
Make sure feature versions listed above match those used for any other dataset in the same project


./lpne-data-analysis/data_tools.py:144: UserWarning: Raw Granger causality values loaded; if you want exponentiated or capped features, you must do that yourself
  warnings.warn("Raw Granger causality values loaded; if you want exponentiated or capped features, you must do that yourself")


In [3]:
y_mouse_aw = np.array(labels['allWindows']['mouse'])
y_time_aw = np.array(labels['allWindows']['time'])
y_expDate_aw = np.array(labels['allWindows']['expDate'])
y_roi_aw = np.zeros(y_expDate_aw.shape) *np.nan
y_hc_aw = np.zeros(y_expDate_aw.shape)*np.nan

y_mouse_w = np.array(labels['windows']['mouse'])
y_time_w = np.array(labels['windows']['time'])
y_expDate_w = np.array(labels['windows']['expDate'])
y_roi_w = np.zeros(y_expDate_w.shape) *np.nan
y_hc_w = np.zeros(y_expDate_w.shape)*np.nan


In [5]:
behavior_files = os.listdir(LABELS_PATH)
mouse_start_times = {}

for file in behavior_files:
    df = pd.read_excel(LABELS_PATH + file)
    start_time = df.StartTime[0]
    video_end = df.Zone.shape[0]
    
    mouse, date = file.split('_')[:2]
    mouse_mask = y_mouse_aw == mouse
    expDate_mask = y_expDate_aw == date
    time_mask = np.logical_and(y_time_aw >= start_time, y_time_aw < start_time + video_end)
    mouse_start_times[mouse] = start_time

    hc_mask = y_time_aw < start_time
    aw_mask = np.logical_and(time_mask,mouse_mask)
    
    y_roi_aw[aw_mask==1] = df.Zone.values
    y_hc_aw[mouse_mask==1] = hc_mask[mouse_mask==1]
    
    #filter windows
    mouse_mask_w = y_mouse_w==mouse
    time_idxs = y_time_w[mouse_mask_w==1] - 1
    mouse_y_roi_aw = y_roi_aw[mouse_mask==1]
    mouse_y_hc_aw = y_hc_aw[mouse_mask==1]
    
    y_roi_w[mouse_mask_w==1] = mouse_y_roi_aw[time_idxs]
    y_hc_w[mouse_mask_w==1] = mouse_y_hc_aw[time_idxs]
    print(start_time)
    print(np.sum(np.logical_and(time_mask,mouse_mask)),df.Zone.shape)
    print(np.unique(df.Zone,return_counts=True))

340.0
788 (788,)
(array([1, 2, 3, 4, 5]), array([125,  10, 278,  30, 345]))
323.0
668 (668,)
(array([1, 2, 3, 4, 5]), array([161,   2, 150,  26, 329]))


In [95]:
save_dict  = {
    "X" :np.hstack([X_psd*10,X_coh,X_gc]),
    "X_psd":X_psd,
    "X_coh":X_coh,
    "X_gc":X_gc,
    "y_mouse":y_mouse_w,
    "y_expDate":y_expDate_w,
    "y_time":y_time_w,
    "y_roi":y_roi_w,
    "y_hc":y_hc_w
}

#with open(SAVE_FILE,"wb") as f:
#    pickle.dump(save_dict,f)

In [5]:
df.Zone

0      3
1      3
2      3
3      3
4      3
      ..
663    5
664    5
665    5
666    5
667    5
Name: Zone, Length: 668, dtype: int64

### Behavior

In [6]:
mouse_list = []
counts_list = []
all_rois = np.array([1.,2.,3.,4.,5.,np.nan])
for mouse in np.unique(y_mouse_aw):
    mouse_list.append(mouse)
    mouse_mask = y_mouse_aw==mouse
    time_mask = np.logical_and(y_time_aw >= mouse_start_times[mouse],
                               y_time_aw < mouse_start_times[mouse] + 300)
    
    mask = np.logical_and(mouse_mask,time_mask)
    
    temp_rois, temp_counts = np.unique(y_roi_aw[mask==1],return_counts=True)
    
    temp_second_rois = [temp_counts[i-1] if i in temp_rois else 0 for i in range(1,6)]
    if np.isnan(temp_rois[-1]): 
        temp_second_rois.append(temp_counts[-1])
    else:
        temp_second_rois.append(0)
    
    counts_list.append(temp_second_rois)
    print(mouse, temp_second_rois)

Mouse67151 [52, 2, 71, 21, 154, 0]
Mouse67152 [59, 7, 100, 25, 109, 0]


In [7]:
roi_matrix = np.vstack(counts_list)
closed_arm_times = roi_matrix[:,0] + roi_matrix[:,2]
open_arm_times = roi_matrix[:,1] + roi_matrix[:,3]
center_times = roi_matrix[:,4]
nan_times = roi_matrix[:,5]

df_behavior = pd.DataFrame.from_dict({"mouse":mouse_list,
                                      "closed_arm_time":closed_arm_times,
                                      "open_arm_time":open_arm_times,
                                      "center_time":center_times,
                                      "nan_time":nan_times})

In [8]:
df_behavior

,mouse,closed_arm_time,open_arm_time,center_time,nan_time
0,Mouse67151,123,23,154,0
1,Mouse67152,159,32,109,0


In [9]:
df_behavior.to_csv("../Projections/Apr_2024_CLOCK_EPM_ROI_durations.csv",index=False)